In [1]:
import pandas as pd
import os
import shutil
import sys
import numpy
import sklearn
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, auc, confusion_matrix, f1_score, precision_score, roc_curve

In [2]:
import time
t = time.time()

In [3]:
infile = "/repos/tables/glom_xml_split.tab"
df = pd.read_table(infile, usecols=["file_id", "split"])
# df['png'] = df.file_id.map(lambda x: x+".png")
df = df.set_index("file_id")["split"]
df.shape

(83,)

In [4]:
# df.set_index('id', inplace=True)

In [58]:
indir = "/repos/data/glom/data_1024/glom_split//all"
outdir = os.path.dirname(indir.rstrip('/'))
def datagen(indir):
    for dd in os.scandir(indir):
        for ff in os.scandir(dd.path):
            if os.path.isdir(ff.path) or not ff.name.endswith("png"):
                continue
#             print(ff.name.split('-')[0], ff.path)
            yield (ff.name.split('-')[0], ff)
    
def gen_set(indir, outdir, df):
    for slideid, ff in datagen(indir):
        yield ff.path, os.path.join(outdir, set_, ff.name)

In [59]:
gen = gen_set(indir, outdir, df)
# sum((1 for _ in gen))

In [60]:
for _, set_ in df.drop_duplicates().items():
    setdir = os.path.join(outdir, set_)
    print(setdir)
    os.makedirs(setdir, exist_ok=True)

/repos/data/glom/data_1024/glom_split/train
/repos/data/glom/data_1024/glom_split/test
/repos/data/glom/data_1024/glom_split/val


In [61]:
for nn, (ifn, ofn) in enumerate(gen):
    try:
        os.symlink(ifn, ofn)
    except FileExistsError:
        continue
    if nn % 1000 == 0:
        print(nn)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
